In [22]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

from   datetime          import timedelta
from   datetime          import datetime


import pandas            as     pd

dt = datetime.now()

from   io                import StringIO

from arable_data_dict import Calibrated_Data_Columns, Hourly_Data_Columns, Daily_Data_Columns

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

start_current_month = ym_to_dt(dt_to_ym(dt)).replace(day=1)
print(start_current_month)
print(dt_to_ym(start_current_month))

today = dt

import os
# dirpath = 'C:/Users/ccber-arables/Box/'
dirpath = 'C:/Users/15303/Documents/'

def write_csv_file(sta, end, data_type, data_type_fn, folder, cm, m, device): 
    logfile = 'Log_' + str(dt_to_ymd(today)) + '.txt'
    devicee = "['" + str(device) + "']"
    #col = ['sdi12_version', 'sdi12_vendor_id', 'sdi12_sensor_model', 'sdi12_sensor_version', 'sdi12_sensor_sn']
    fn = str(folder) + str(devicee)[2:-2] + '_' + str(cm) + '_' + str(data_type_fn) + '.csv'
    print(fn)
    try:
        df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = devicee, 
                     measure = str(data_type), 
                     order = "time", 
                     end = end, 
                     start = sta, 
                     limit = 100000000)
        df = StringIO(df)
        df = pd.read_csv(df, sep=',', error_bad_lines=False)
        df = df.dropna(axis='columns', thresh=2)#how='all')
        df = df.drop('location', 1)
        df = df.rename(mapper=m)
        df.to_csv(fn, sep = ",")
        print('Successfully wrote ' + fn)
        open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
    except:
        open(logfile, 'a+').write('Failure writing ' + fn + '\n')
        pass  

    
    
    
device_ids = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['ID'].values.tolist()    
print(device_ids) 
locations  = pd.read_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")['Location'].values.tolist()  
print(locations)

# ---> YEAR
# --------> MONTH
# --------------> DATA TYPE
# ------------------------> DEVICE-ID_MONTH-YEAR_DATA-TYPE.csv
Folder_Name          = start_current_month.strftime("%Y")
print(Folder_Name)
# Sub_Folder_Name      = start_current_month.strftime("%m")
Sub_Sub_Folder_Names = ["Daily", "Hourly", "Soil", "Calibrated"]
Data_Types           = ["daily", "hourly", "aux_raw", "health"]
Column_Names         = [Calibrated_Data_Columns, Hourly_Data_Columns, Hourly_Data_Columns, Daily_Data_Columns]

Folder_Name_B        = [str(str(Sub_Sub_Folder_Names[i]) + '/' + str(Folder_Name) + '/') for i in range(len(Sub_Sub_Folder_Names))]


for i in range(len(locations)):
    Folder_Name_A = str(dirpath + str(locations[i]) + '/')
    for j in range(len(Folder_Name_B)):
        Folder_Name = str(Folder_Name_A + Folder_Name_B[j])
        print(Folder_Name)

'''

for j in range(len(Data_Types)):
    if not os.path.exists(Folder_Names[j]):
        os.makedirs(Folder_Names[j])
        print "Directory " , Folder_Names[j] ,  " Created "
    else:    
        print "Directory " , Folder_Names[j] ,  " already exists"  
        pass
    for dev in device_ids:
        print(dev)
        write_csv_file(start_current_month, today, Data_Types[j], Sub_Sub_Folder_Names[j], Folder_Names[j], dt_to_ym(start_current_month), Column_Names[j], dev)
        
    '''

2020-04-01 00:00:00
2020-04
['A000536', 'A000563', 'A000507', 'A000466', 'A000369', 'A000318', 'A000522', 'A000546', 'A000465', 'A000229', 'A000513']
[' Lagoon 2016 Burn', ' Stipa Loam NCOS', ' BlackLakeGentleSo', ' NCOS Saltmarsh', ' SaltMarshMilkVetch', ' LagoonCoastalSage', ' Philips 66 Nipomo', ' NCOS 2017 Stipa', ' CampusPtIceplant', ' LagoonAnnualGrass', ' BlackLakeN-facing']
2020
C:/Users/15303/Documents/ Lagoon 2016 Burn/Daily/2020/
C:/Users/15303/Documents/ Lagoon 2016 Burn/Hourly/2020/
C:/Users/15303/Documents/ Lagoon 2016 Burn/Soil/2020/
C:/Users/15303/Documents/ Lagoon 2016 Burn/Calibrated/2020/
C:/Users/15303/Documents/ Stipa Loam NCOS/Daily/2020/
C:/Users/15303/Documents/ Stipa Loam NCOS/Hourly/2020/
C:/Users/15303/Documents/ Stipa Loam NCOS/Soil/2020/
C:/Users/15303/Documents/ Stipa Loam NCOS/Calibrated/2020/
C:/Users/15303/Documents/ BlackLakeGentleSo/Daily/2020/
C:/Users/15303/Documents/ BlackLakeGentleSo/Hourly/2020/
C:/Users/15303/Documents/ BlackLakeGentleSo/Soil/20

'\n\nfor j in range(len(Data_Types)):\n    if not os.path.exists(Folder_Names[j]):\n        os.makedirs(Folder_Names[j])\n        print "Directory " , Folder_Names[j] ,  " Created "\n    else:    \n        print "Directory " , Folder_Names[j] ,  " already exists"  \n        pass\n    for dev in device_ids:\n        print(dev)\n        write_csv_file(start_current_month, today, Data_Types[j], Sub_Sub_Folder_Names[j], Folder_Names[j], dt_to_ym(start_current_month), Column_Names[j], dev)\n        \n    '

In [10]:
import pandas as pd

    df = pd.DataFrame.from_dict(dictionary, orient='index', dtype='string', columns=[colname])
    return df

CalibratedDF       = importcolnames(Calibrated_Data_Columns, 'Name')
HourlyDF           = importcolnames(Hourly_Data_Columns, 'Name')
DailyDF            = importcolnames(Daily_Data_Columns, 'Name')

CalibratedDF_Units = importcolnames(Calibrated_Data_Columns_Units, 'Units')
HourlyDF_Units     = importcolnames(Hourly_Data_Columns_Units, 'Units')
DailyDF_Units      = importcolnames(Daily_Data_Columns_Units, 'Units')


                                                         Name
tair_version                                          UNKNOWN
B1uw                      Spectrometer data Bands 1 upwelling
LWdw                       Long wave downwelling, hourly mean
Tair                                          Air temperature
SWuw                        Short wave upwelling, hourly mean
Tabove                                        Sky temperature
Tbelow                                Leaf/Ground temperature
B5uw                      Spectrometer data Bands 1 upwelling
RH                                          Relative humidity
B7uw                      Spectrometer data Bands 1 upwelling
B3dw                    Spectrometer data Bands 1 downwelling
B5dw                    Spectrometer data Bands 1 downwelling
PARdw         Photosynthetically Active Radiation downwelling
B2uw                      Spectrometer data Bands 1 upwelling
B3uw                      Spectrometer data Bands 1 upwelling
PARuw   